# AI FAQ 챗봇 만들기 with RAG

In [6]:
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.schema import Document
import json

1. 데이터 로드

In [8]:
with open('./data/faq_chatbot_data.json', 'r', encoding='utf-8') as f:
    faq_data = json.load(f)

In [9]:
faq_data

[{'question': '반품 정책이 어떻게 되나요?',
  'answer': '제품을 수령한 후 14일 이내에 반품이 가능합니다. 반품 신청은 고객센터에서 도와드립니다.'},
 {'question': '배송은 얼마나 걸리나요?',
  'answer': '일반적으로 2~5일 정도 소요되며, 지역에 따라 다를 수 있습니다.'},
 {'question': '회원가입 없이 주문할 수 있나요?',
  'answer': '네, 비회원 주문이 가능합니다. 다만, 주문 조회 및 혜택을 받기 위해 회원가입을 추천드립니다.'},
 {'question': '포인트 적립은 어떻게 되나요?',
  'answer': '구매 금액의 1%가 적립되며, 특별 프로모션 기간 동안 추가 적립이 가능합니다.'},
 {'question': '해외 배송이 가능한가요?',
  'answer': '일부 국가를 대상으로 해외 배송이 가능하며, 배송비는 지역에 따라 상이할 수 있습니다.'},
 {'question': '제품 보증 기간은 얼마나 되나요?',
  'answer': '제품 보증 기간은 기본적으로 1년이며, 일부 제품은 연장 보증이 가능합니다.'},
 {'question': 'A/S 신청은 어떻게 하나요?',
  'answer': '공식 홈페이지 또는 고객센터를 통해 A/S 신청이 가능합니다.'},
 {'question': '재고가 없는 상품은 언제 다시 입고되나요?',
  'answer': '입고 일정은 상품마다 다르며, 알림 설정을 통해 입고 시 안내받을 수 있습니다.'},
 {'question': '구매 후 영수증을 받을 수 있나요?',
  'answer': '네, 주문 내역에서 전자 영수증을 다운로드할 수 있습니다.'},
 {'question': '다른 사람이 대신 주문을 수령할 수 있나요?',
  'answer': '네, 수령인을 지정할 수 있으며 신분증 확인 후 수령 가능합니다.'},
 {'question': '결제 방법에는 어떤 것이 있나요?',
  'answer': '신용카드, 

2. embedding 모델 생성 + vector store 생성

In [11]:
embedding_model = OpenAIEmbeddings()
# print(embedding_model.model)

vector_store = Chroma(
    embedding_function=embedding_model,
    persist_directory='./chroma_db'
)

documents = [Document(page_content=item['question']+" "+item['answer']) for item in faq_data]
vector_store.add_documents(documents)

C:\Users\Playdata\AppData\Local\Temp\ipykernel_6052\1749445865.py:4: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_store = Chroma(


['1adf2cb1-c491-4a27-a5a3-96c4d1a80cec',
 '047ac89b-5a3a-4c37-94f4-288be641bbb2',
 'e606648b-cc65-4ba4-a652-e9f86dfe0647',
 'f809a819-574c-4eeb-95f6-d1298a9e5c87',
 '849c7d0a-7e1c-49d9-b203-a32ff6985d6e',
 '5d5d624c-539f-4f03-a02e-78112f737162',
 'c83210b1-0b38-45ce-b8f8-9caf44469365',
 'a927c9bf-0c97-4295-a502-11038cca6c52',
 '5a151a1c-43fa-46a8-aa55-2c0a0748860e',
 'ce578699-22a1-4d5e-ba56-f5b68dca3492',
 'cfa6ffe5-6e26-48fb-8b93-57f2919cd210',
 '276bc291-6a99-4a57-bd3a-1c2e011dfe2b',
 '07c7e1da-ac1b-4137-8ab8-7e914ce788f2']

In [ ]:
embedding_model = OpenAIEmbeddings()
# print(embedding_model.model)

documents = [Document(page_content=item['question']+" "+item['answer']) for item in faq_data]

vector_store = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    persist_directory='./chroma_db'
)

3. llm 설정 (모델 생성)

In [12]:
model = ChatOpenAI(model='gpt-4o-mini', temperature=0.5)

4. 프롬프트 및 체인 설정

In [ ]:
prompt = PromptTemplate(
    template="""
다음 '질문'에 대해 '문서'의 내용만을 참고하여 '답변'을 생성해 주세요.
질문 : {question} 
문서 : {context}
답변 :
""",
    input_variables=['question', 'context']
)

# prompt를 구성할 때 retriever의 검색 결과로 들어갈 input_variable은 'context'로 설정해야 함

In [14]:
retriever = vector_store.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x000002E97FC5B910>, search_kwargs={})

In [17]:
qa_chain = RetrievalQA.from_chain_type(
    llm=model,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt}
)

5. 사용

In [18]:
qa_chain.run('배송 얼마나 걸려?')

C:\Users\Playdata\AppData\Local\Temp\ipykernel_6052\834878008.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa_chain.run('배송 얼마나 걸려?')


'배송은 일반적으로 2~5일 정도 소요되며, 지역에 따라 다를 수 있습니다.'

In [19]:
qa_chain.run('구매 취소하고 싶은데')

'상품이 출고되지 않은 상태라면 주문 취소가 가능합니다.'

In [20]:
qa_chain.run('구매 영수증을 발급해줘')

'네, 주문 내역에서 전자 영수증을 다운로드할 수 있습니다.'